In [10]:
import csv
import collections
import statistics

In [135]:
with open('/global/scratch/users/pierrj/eccDNA/magnaporthe_pureculture/rawdata/illumina/G3_1A_tests/ecc_confidence') as coverage:
    coverage_reader = csv.reader(coverage, delimiter = '\t')
    coverage_list = [[] for i in range(56)]
    for row in coverage_reader:
        coverage_list[int(row[0])].append([int(row[1]), int(row[2]), str(row[3]), str(row[4]), str(row[5])])

In [ ]:
## name eccs (Chrom[num]ecc[num])
## maybe do this after actually

In [136]:
def uniq(lst):
    last = object()
    for item in lst:
        if item == last:
            continue
        yield item
        last = item

def sort_and_deduplicate(l):
    return list(uniq(sorted(l, key = lambda x: x[0])))

In [137]:
duplicate_test_list = [[2999, 3499], [3000, 3500], [3001, 3501], [3051, 3551], [3052, 3552], [3053, 3553]]
sortuniqcoverage_list_chrom0 = sort_and_deduplicate(coverage_list[0])
# sortuniqcoverage_list_chrom0 = duplicate_test_list

In [116]:
print(sortuniqcoverage_list_chrom0)

[[2999, 3499], [3000, 3500], [3001, 3501], [3051, 3551], [3052, 3552], [3053, 3553]]


In [162]:
ecc_withvariants = {}
for i in range(len(sortuniqcoverage_list_chrom0)):
    ecc = sortuniqcoverage_list_chrom0[i]
    variants = []
## pretty major issue is that this does a rolling thing so some eccs probably appear twice
    for k in range(len(sortuniqcoverage_list_chrom0)):
        start_coordinate1 = sortuniqcoverage_list_chrom0[k][0] - 50
        start_coordinate2 = sortuniqcoverage_list_chrom0[k][0] + 50
        end_coordinate1 = sortuniqcoverage_list_chrom0[k][1] - 50
        end_coordinate2 = sortuniqcoverage_list_chrom0[k][1] + 50
        if start_coordinate1 <= ecc[0] <= start_coordinate2 and end_coordinate1 <= ecc[1] <= end_coordinate2:
            variants.append(sortuniqcoverage_list_chrom0[k])
    if tuple([ecc]) != tuple(variants): ## if there isn't just itself in the list
        ecc_withvariants[tuple(ecc)] = variants
##can probably look for duplicates here
variants_grouped = collections.defaultdict(list)
for key,val in ecc_withvariants.items(): ## group eccs with same variants
    #variants_grouped[tuple(val)].append(key)
    variants_grouped[tuple(tuple(x) for x in val)].append(key)
    ## shouldn't be for x in val
    ## grouping isn't happening exactly the way I think it is

In [128]:
print(ecc_withvariants)

{(2999, 3499): [[2999, 3499], [3000, 3500], [3001, 3501]], (3000, 3500): [[2999, 3499], [3000, 3500], [3001, 3501]], (3001, 3501): [[2999, 3499], [3000, 3500], [3001, 3501], [3051, 3551]], (3051, 3551): [[3001, 3501], [3051, 3551], [3052, 3552], [3053, 3553]], (3052, 3552): [[3051, 3551], [3052, 3552], [3053, 3553]], (3053, 3553): [[3051, 3551], [3052, 3552], [3053, 3553]]}


In [129]:
print(variants_grouped)

defaultdict(<class 'list'>, {((2999, 3499), (3000, 3500), (3001, 3501)): [(2999, 3499), (3000, 3500)], ((2999, 3499), (3000, 3500), (3001, 3501), (3051, 3551)): [(3001, 3501)], ((3001, 3501), (3051, 3551), (3052, 3552), (3053, 3553)): [(3051, 3551)], ((3051, 3551), (3052, 3552), (3053, 3553)): [(3052, 3552), (3053, 3553)]})


In [67]:
def getcoord(coord_list): ## get ideal representative coordinates based off medians
    median = statistics.median(coord_list)
    distancetoend = abs(median - max(coord_list))
    distancetostart = abs(median - min(coord_list))
    greaterlist = [i for i in coord_list if i >= median]
    smallerlist = [i for i in coord_list if i <= median]
    #if median is closer to end, get value above median. if median is closer to start, get value below median
    #if equal get both values, one as alt
    if distancetoend > distancetostart:
        coord = min(smallerlist, key=lambda x:abs(x-median))
        coord_alt = 'N/A'
    elif distancetoend == distancetostart:
        coord = min(greaterlist, key=lambda x:abs(x-median))
        coord_alt = min(smallerlist, key=lambda x:abs(x-median))
    elif distancetoend < distancetostart:
        coord = min(greaterlist, key=lambda x:abs(x-median))
        coord_alt = 'N/A'
    else:
        raise ValueError("getcoord error")
    return coord, coord_alt

In [68]:
def test_coords(target_coord, other_coord, target_start_or_end): ## get closest matching coordinate based off on set/forced coordinate
        ## check if start or end coordinate is the set one
        if target_start_or_end == 'start':
            known = 0
            unknown = 1
        if target_start_or_end == 'end':
            known = 1
            unknown = 0
        ## get all potential options matching set coordinate
        unknown_options = []
        for i in val:
            if i[known] == target_coord:
                unknown_options.append(i[unknown])
        ## test which one is the closest to the ideal median
        unknown_coord = min(unknown_options, key=lambda x:abs(x-other_coord))
        return unknown_coord

def reconcile_coords(startcoord, endcoord): ## test which coordinate pair is closest to ideal
        #test with start coordinate forced
        startcoord_forced = test_coords(startcoord, endcoord, 'start')
        #test with end coordinate forced
        endcoord_forced = test_coords(endcoord, startcoord, 'end')
        #test how close each non-ideal corresponding coordinates are to the ideal
        fit_startcoord_forced = abs(startcoord_forced - endcoord)
        fit_endcoord_forced = abs(endcoord_forced - startcoord)
        if fit_startcoord_forced <= fit_endcoord_forced:
            return startcoord, startcoord_forced
        else:
            return endcoord_forced, endcoord

In [163]:
representative_variants = {}
for key, val in variants_grouped.items():
    startlist = []
    endlist = []
    for i in val:
        startlist.append(i[0])
        endlist.append(i[1])
    start_coord, start_coord_alt = getcoord(startlist)
    end_coord, end_coord_alt = getcoord(endlist)
    middle_ecc = 0
    ## check to see if a single ecc has ideal coordinates for both end and start
    for i in val:
        if i[0] == start_coord and i[1] == end_coord:
            middle_ecc = i
        if i[0] == start_coord and i[1] == end_coord_alt:
            middle_ecc = i
        if i[0] == start_coord_alt and i[1] == end_coord:
            middle_ecc = i
        if i[0] == start_coord and i[1] == start_coord_alt:
            middle_ecc = i
    ## if not then reconcile coordinates
    if middle_ecc == 0:
        start_coord_estimate, end_coord_estimate = reconcile_coords(start_coord, end_coord)
        for i in val:
            if i[0] == start_coord_estimate and i[1] == end_coord_estimate:
                middle_ecc = i
    if middle_ecc == 0:
        raise ValueError("No ecc merged ecc called")
    else:
        representative_variants[middle_ecc] = val

In [140]:
print(representative_variants)

{(3269, 3771, 'conf', 'coverage_too_low', '39.61;54.29;29.86'): [(3269, 3796, 'conf', 'coverage_too_low', '38.35;54.12;29.12'), (3269, 3809, 'conf', 'coverage_too_low', '37.7;53.82;28.9'), (3269, 3767, 'conf', 'coverage_too_low', '39.82;54.31;29.96'), (3269, 3771, 'conf', 'coverage_too_low', '39.61;54.29;29.86')], (4158, 6063, 'conf', 'coverage_too_low', '32.96;94.78;109.82'): [(4158, 6073, 'conf', 'coverage_too_low', '32.83;95.17;109.15'), (4158, 6063, 'conf', 'coverage_too_low', '32.96;94.78;109.82')], (4929, 5720, 'conf', 'coverage_too_low', '35.05;137.71;132.46'): [(4929, 5737, 'conf', 'coverage_too_low', '34.74;137.28;133.04'), (4929, 5720, 'conf', 'coverage_too_low', '35.05;137.71;132.46')], (5068, 8394, 'hconf', 'none', '31.52;113.92;24.0'): [(5068, 8394, 'hconf', 'none', '31.52;113.92;24.0'), (5068, 8408, 'hconf', 'none', '31.41;113.69;23.87'), (5068, 8386, 'hconf', 'none', '31.58;114.05;24.05')], (12244, 13704, 'conf', 'coverage_too_low', '18.85;126.55;64.09'): [(12244, 13712,

In [183]:
singlets_list = []
for key, val in representative_variants.items():
    if len(val) == 30:
        print(val)
        print(key)
        singlets_list.append(val)

[(1017984, 1018294, 'hconf', 'none', '137.64;869.14;341.55'), (1017985, 1018288, 'hconf', 'none', '137.55;871.43;360.08'), (1017985, 1018318, 'hconf', 'none', '138.1;870.34;276.16'), (1017985, 1018312, 'hconf', 'none', '137.98;870.64;291.99'), (1017985, 1018326, 'hconf', 'none', '138.27;870.25;255.66'), (1017985, 1018299, 'hconf', 'none', '137.75;871.18;327.9'), (1017985, 1018320, 'hconf', 'none', '138.13;870.29;270.96'), (1017985, 1018284, 'hconf', 'none', '137.43;871.28;372.6'), (1017985, 1018322, 'hconf', 'none', '138.17;870.26;265.78'), (1017985, 1018282, 'hconf', 'none', '137.37;871.19;379.04'), (1017985, 1018280, 'hconf', 'none', '137.29;871.09;385.62'), (1017985, 1018311, 'hconf', 'none', '137.96;870.7;294.68'), (1017985, 1018297, 'hconf', 'none', '137.72;871.29;333.57'), (1017985, 1018317, 'hconf', 'none', '138.08;870.37;278.8'), (1017985, 1018328, 'hconf', 'none', '138.34;870.26;250.71'), (1017985, 1018306, 'hconf', 'none', '137.86;870.97;308.34'), (1017985, 1018329, 'hconf', 

In [202]:
print(len(sortuniqcoverage_list_chrom0))
print(len(ecc_withvariants))
print(len(representative_variants))
print(len(singlets_list))
print(len(premerge))
print(len(postmerge))

12930
8378
2483
1
12930
2483


In [200]:
premerge = []
for i in range(len(sortuniqcoverage_list_chrom0)):
    to_add = []
    to_add.append('MQOP01000001.1')
    to_add.append(sortuniqcoverage_list_chrom0[i][0])
    to_add.append(sortuniqcoverage_list_chrom0[i][1])
    premerge.append(to_add)

postmerge = []
for key in representative_variants.keys():
    to_add = []
    to_add.append('MQOP01000001.1')
    to_add.append(key[0])
    to_add.append(key[1])
    postmerge.append(to_add)

with open('/global/home/users/pierrj/testfiles/test_premerge', 'w', newline = '') as premerge_file:
    w = csv.writer(premerge_file, delimiter = '\t')
    w.writerows(premerge)

with open('/global/home/users/pierrj/testfiles/test_postmerge', 'w', newline = '') as postmerge_file:
    w = csv.writer(postmerge_file, delimiter = '\t')
    w.writerows(postmerge)

In [61]:
def check_duplicates(dictionary):
    check_list = []
    for key, value in dictionary.items(): 
        check_list.append(value)
    flat_list = [item for sublist in check_list for item in sublist]
    if len(flat_list) == len(set(flat_list)):
        return True
    else:
        raise ValueError('duplicate eccs found in ecc_merge')

## this doesn't work because values are whole lists in stead of each item

## add all values in a dictionary to a list of lists
## turn list of lists in list
## then check for uniques

In [204]:
print(check_duplicates(representative_variants))

True


In [ ]:
## loop through items in the list
## replace them with the key if they are
## nvm that's super duper slow
## how about just deleting them from the list when you pick them out
## then add the keys back into the list
for k, v in BIBLE_BOOKS_DICT.items():
    if input in v:
        print(k)

In [ ]:
### count how many eccs are within 100 bp of them on either side
### pick the ones with the highest counts somehow?
## something about the length?
## make a dictonary where the key is the ecc and the values are the lists that ecc is next to, including itself
## then delete keys with duplicate values
## then keep key that is in the middle